In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 문서 가져오기
pdf_path = "../00_data/Sustainability_report_2024_kr.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()
len(docs[:20])

20

In [ ]:
# 2. 청크 나누기
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
chunks = splitter.split_documents(docs[:20])
print(len(chunks))

48


In [5]:
# 3. 벡터 스토어 및 retriever 구성하기
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate

embeddings = OpenAIEmbeddings()
db_path = "../07_vectorstore/rag_eval_20"
vectorstore = Chroma.from_documents(
    documents = chunks,
    embedding = embeddings,
    persist_directory = db_path,
    collection_name = "samsung2024_eval"
)

In [6]:
# 4. retriever 구성하기
retriever = vectorstore.as_retriever(
    search_kwargs = {"k" : 5}
)

In [11]:
# 5. 프롬프트 구성하기
system_prompt = (
    "You are a helpful assistant. Answer strictly based on the provided context. "
    "If the answer is not in the context, say you don't know."
    "context : {context}"
)
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

In [7]:
# 6. 모델 구성하기
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0
)

In [8]:
# 7. 아웃풋 파서
from langchain_core.output_parsers import StrOutputParser
outputparser = StrOutputParser()

In [14]:
# 8. 체인 설정
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# 문서 합치는 함수 설정
def format_docs(docs):
    return "\n\n---\n\n".join(d.page_content for d in docs)

# 체인 만들기
rag_chain = (
    {"context" : RunnableLambda(lambda x : x["question"]) | retriever | format_docs,
     "question" : RunnablePassthrough()
     }
     | rag_prompt
     | model
     | outputparser
)

In [15]:
rag_chain.invoke({"question":"삼성전자의 전망은?"})

'삼성전자는 어려운 대내외 환경 속에서도 역대 최고 수준의 연구개발 투자(28.3조원)와 전략적 시설투자(53.1조원)를 통해 기술 리더십을 강화하며 중장기 수요에 미리 대응하고 있습니다. 또한, 지속가능경영을 기업의 방향성과 사업 경쟁력, 기술 혁신의 원동력으로 삼아 새로운 도약을 모색하고 있으며, 환경, 사회, 경제적 리스크와 지정학적 불확실성 속에서도 지속 가능한 미래를 위해 최선을 다할 계획입니다.'

In [16]:
# 9. 평가용 데이터 불러오기
import pandas as pd
excel_path = "report_2024_test.xlsx"
df = pd.read_excel(excel_path)
df.head()

,user_input,reference_contexts,reference,synthesizer_name
0,독일에서 2023년에 발효된 공급망 관련 법은 무엇인가요?,['Facts & Figures PrinciplePlanet People\nCEO ...,독일에서는 공급망의 인권과 근로환경 관리를 의무화하는 공급망실사법이 2023년에 발...,single_hop_specific_query_synthesizer
1,Could you please explain the role and signific...,"[""접수된 고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 ...",희망별숲 is a subsidiary company established in Ma...,single_hop_specific_query_synthesizer
2,Could you explain the role of the 글로벌 행동규범 in ...,['삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendi...,Samsung Electronics has established the 글로벌 행동...,single_hop_specific_query_synthesizer
3,What is the role of 메모리 반도체 in Samsung Electro...,['Our Company AppendixMateriality Assessment F...,메모리 반도체 사업은 삼성전자의 DS(Device Solutions) 부문에 속하며...,single_hop_specific_query_synthesizer
4,What DS Device Solutions mean in context of 20...,['Device eXperienceDX\n DS Device Solutions\n메...,DS Device Solutions refers to a segment mentio...,single_hop_specific_query_synthesizer


- user_input : 질문
- reference_contexts : 예상되는 답변을 만들기위해 참고한 contexts
- reference : 예상되는 답변
***
- user_input : 질문
- retrieved_contexts : 검색한 자료 -> 리스트
- response : 실제 답변

In [18]:
# for 문으로 구현하기
answer = []
contexts = []
for question in df["user_input"]:
    docs = retriever.invoke(question)
    ctx_list = []
    for d in docs:
        ctx_list.append(d.page_content)
    contexts.append(ctx_list)

In [19]:
contexts

[["대한 관심 또한 지속 고조되고 있습니다. \n삼성전자는 이러한 추세에 맞춰 지속가능한 미래를 위한 노력을 계속해 \n왔습니다. 2050년 탄소중립을 통해 글로벌 기후위기 극복 노력에 \n동참하고, 자원의 순환성을 극대화하여 순환경제 구축 에 기여하며, \n기술혁신을 통해 환경난제 해결에 도전하는 내용의 '新환경경영전략'을 \n2022년 9월에 발표하였습니다. \n2030년 탄소중립을 목표로 하는 DX부문에서는 미국, 유럽, 중국에 이어 \n한국, 베트남, 인도, 브라 질을 포함한 글로벌 주요 제조사업장의 사용 \n전력을 재생에너지로 100% 전환하였으며, 2023년말 기준, 총 93.4%\n의 재생에너지 전환 실적을 기록하였습니다. 또한 자원순환형 소재 확대 \n로드맵에 맞춰, 당사가 구매한 제품 내 플라스틱 부품 25%에 재생레진을 \n적용하였습니다. \nDS부문에서는 2050년 탄소중립 달성을 위해 2023년 공정가스 처리시설\n(RCS) 16대를 4개 라인에 신규 설치하고, LNG 폐열 회수 시스템을 확대 \n적용하여 온실가스 배출 저감을 위해 노력했습니다. 또한 2030년까지 \n용수 취수량을 2021년 수준으로 절 감하는 것을 목 표로 하고 있으며, \n국제수자원관리동맹(AWS)으로부터 기흥/화성, 평택, 중국 시안 사업장에 \n대해 최고 등급인 ' 플래티넘' 인증을 획득하여 수자원 관리 체계의 \n우수성을 입증 받았습니다. 아울러, 폐기물 발생량 저감을 위해 2024년 \n3월, 웨이퍼박스 등 9건에 대해 환경부 순환자원 인정을 추가 취득하여 \n현재까지 총 49건을 순환자원으로 인정받았고 매년 품목을 확대해나가고 \n있습니다.\n사회 분야에서는 지난 2023년 2월 발표한 글로벌 인권원칙에 연계하여, \n글로벌 고충처리 정책 을 2024년 4월 제정, 고충  접수 채널 운영과 \n접수된 고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 비제조 분야 및 리스크 분석에 따라 제조 분야 2차 협력회사로",
  '기회요인으로 예

In [21]:
for question in df["user_input"]:
    an = rag_chain.invoke({"question" : question})
    answer.append(an)

In [22]:
answer[:5]

['독일에서 2023년에 발효된 공급망 관련 법은 "공급망실사법"입니다.',
 "The subsidiary company 희망별숲, established by Samsung Electronics in March 2023 as a standardized workplace for individuals with disabilities, plays a significant role in providing stable employment opportunities for people with developmental disabilities. 희망별숲 produces bakery products such as cookies and muffins, which are supplied to Samsung Electronics employees. Starting with the bakery sector, Samsung Electronics plans to expand stable job opportunities for individuals with developmental disabilities through this subsidiary. This initiative reflects Samsung Electronics' commitment to social responsibility and inclusive employment, contributing to sustainable management and community support.",
 'The provided context does not contain specific information about the "글로벌 행동규범" (Global Code of Conduct) or its role in guiding Samsung Electronics\' management activities and organizational culture. Therefore, I do not have the information to exp

In [23]:
len(answer)

142

In [25]:
fixed_answer = answer[-100:]
len(fixed_answer)

100

In [24]:
len(contexts)

100

In [ ]:
retriever_contexts

In [50]:
df["response"] = fixed_answer
df["retrieved_contexts"] = contexts
df.head()

,user_input,reference_contexts,reference,synthesizer_name,response,retriever_contexts,retrieved_contexts
0,독일에서 2023년에 발효된 공급망 관련 법은 무엇인가요?,[Facts & Figures PrinciplePlanet People\nCEO 메...,독일에서는 공급망의 인권과 근로환경 관리를 의무화하는 공급망실사법이 2023년에 발...,single_hop_specific_query_synthesizer,"독일에서 2023년에 발효된 공급망 관련 법은 ""공급망실사법""입니다.",[대한 관심 또한 지속 고조되고 있습니다. \n삼성전자는 이러한 추세에 맞춰 지속가...,[대한 관심 또한 지속 고조되고 있습니다. \n삼성전자는 이러한 추세에 맞춰 지속가...
1,Could you please explain the role and signific...,"[접수된 고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 비...",희망별숲 is a subsidiary company established in Ma...,single_hop_specific_query_synthesizer,"The subsidiary company 희망별숲, established by Sa...","[접수된 고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 비...","[접수된 고충의 처리 원칙에 대한 기준을 수립하였고, 공급망 관리에 \n있어서는 비..."
2,Could you explain the role of the 글로벌 행동규범 in ...,[삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendix...,Samsung Electronics has established the 글로벌 행동...,single_hop_specific_query_synthesizer,The provided context does not contain specific...,[마련하고 유관 사업 활동을 분석하였습니다. 이후 각 활동과 관련성이 높은 \n밸류...,[마련하고 유관 사업 활동을 분석하였습니다. 이후 각 활동과 관련성이 높은 \n밸류...
3,What is the role of 메모리 반도체 in Samsung Electro...,[Our Company AppendixMateriality Assessment Fa...,메모리 반도체 사업은 삼성전자의 DS(Device Solutions) 부문에 속하며...,single_hop_specific_query_synthesizer,메모리 반도체는 삼성전자 DS(Device Solutions) 부문에 속하는 주요 ...,[삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendix...,[삼성전자 지속가능경영보고서 2024\n05\nOur Company Appendix...
4,What DS Device Solutions mean in context of 20...,[Device eXperienceDX\n DS Device Solutions\n메모...,DS Device Solutions refers to a segment mentio...,single_hop_specific_query_synthesizer,"In the context of 2023 sales and profit, ""DS D...",[DS Device Solutions\n메모리\n※ 상기 매출과 영업이익은 2023...,[DS Device Solutions\n메모리\n※ 상기 매출과 영업이익은 2023...


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_input          100 non-null    object
 1   reference_contexts  100 non-null    object
 2   reference           100 non-null    object
 3   synthesizer_name    100 non-null    object
 4   response            100 non-null    object
 5   retriever_contexts  100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [51]:
df["retrieved_contexts"].dtype

dtype('O')

In [ ]:
## 데이터 타입을 리스트로 변경해야한다.
import ast
df["reference_contexts"] = df["reference_contexts"].apply(lambda x : ast.literal_eval(x))
df["reference_contexts"]

In [45]:
df['reference_contexts'].dtype

dtype('O')

In [52]:
from ragas import EvaluationDataset, evaluate
from ragas.metrics import Faithfulness, LLMContextRecall, FactualCorrectness
from ragas.llms import LangchainLLMWrapper

# ragas 평가
eval_llm = LangchainLLMWrapper(model)
dataset = EvaluationDataset.from_pandas(df)
dataset

C:\Users\user\AppData\Local\Temp\ipykernel_4828\3001053187.py:6: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  eval_llm = LangchainLLMWrapper(model)


EvaluationDataset(features=['user_input', 'retrieved_contexts', 'reference_contexts', 'response', 'reference'], len=100)

In [53]:
scores = evaluate(  # 1.높으면 할루시네이션 낮음, 2. retriever 문제   3. 형식, 시스템 프롬프트
    dataset,
    metrics = [Faithfulness(), LLMContextRecall(), FactualCorrectness()],
    llm = eval_llm
)

Evaluating:   0%|          | 0/300 [00:00<?, ?it/s]

Exception raised in Job[4]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1-mini in organization org-ifMd863rcCIXpEuEYtSj7yxy on requests per min (RPM): Limit 500, Used 500, Requested 1. Please try again in 120ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[7]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1-mini in organization org-ifMd863rcCIXpEuEYtSj7yxy on tokens per min (TPM): Limit 200000, Used 200000, Requested 3052. Please try again in 915ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[1]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1-mini in organization org-ifMd863rcCIXpEuEYtSj7yxy on tokens per min (TPM): Limit 200000, Used 19

In [ ]:
scores